In [37]:
# Start Main Code To Make Model

In [1]:
from pickle import dump

In [2]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = "2"

In [3]:
imgs_source_path = './images/border_only_train/'

In [4]:
all_imgs_list = os.listdir(imgs_source_path)

In [5]:
num_train_imgs = 20
num_test_imgs = 5

In [15]:
class_list = list(set([x.split('.')[-3] for x in all_imgs_list]))
dump(' '.join(class_list), open('class_list.txt', 'wb'))

In [16]:
import random

In [17]:
train_imgs_list = list()
test_imgs_list = list()

for i in range(len(class_list)):
    class_name = class_list[i]
    class_all_imgs_list = [x for x in all_imgs_list if x.split('.')[-3] == class_name]
    class_train_imgs_list = list()
    class_test_imgs_list = list()
    for j in range(num_train_imgs):
        img_name = random.choice(class_all_imgs_list)
        if img_name not in class_train_imgs_list:
            class_train_imgs_list.append(img_name)
            class_all_imgs_list.remove(img_name)
    for j in range(num_test_imgs):
        img_name = random.choice(class_all_imgs_list)
        if img_name not in class_train_imgs_list and img_name not in class_test_imgs_list:
            class_test_imgs_list.append(img_name)
            class_all_imgs_list.remove(img_name)
    train_imgs_list += class_train_imgs_list
    test_imgs_list += class_test_imgs_list
# train_imgs_list, test_imgs_list


(['dinh_lang.30.png',
  'dinh_lang.23.png',
  'dinh_lang.7.png',
  'dinh_lang.26.png',
  'dinh_lang.8.png',
  'dinh_lang.16.png',
  'dinh_lang.28.png',
  'dinh_lang.25.png',
  'dinh_lang.24.png',
  'dinh_lang.10.png',
  'dinh_lang.21.png',
  'dinh_lang.27.png',
  'dinh_lang.19.png',
  'dinh_lang.13.png',
  'dinh_lang.22.png',
  'dinh_lang.31.png',
  'dinh_lang.32.png',
  'dinh_lang.5.png',
  'dinh_lang.18.png',
  'dinh_lang.9.png',
  'tia_to.22.png',
  'tia_to.11.png',
  'tia_to.3.png',
  'tia_to.16.png',
  'tia_to.13.png',
  'tia_to.8.png',
  'tia_to.5.png',
  'tia_to.6.png',
  'tia_to.21.png',
  'tia_to.17.png',
  'tia_to.25.png',
  'tia_to.19.png',
  'tia_to.7.png',
  'tia_to.14.png',
  'tia_to.23.png',
  'tia_to.1.png',
  'tia_to.15.png',
  'tia_to.18.png',
  'tia_to.9.png',
  'tia_to.12.png',
  'co_man_trau.14.png',
  'co_man_trau.27.png',
  'co_man_trau.29.png',
  'co_man_trau.32.png',
  'co_man_trau.8.png',
  'co_man_trau.19.png',
  'co_man_trau.39.png',
  'co_man_trau.13.png',


In [18]:
img_size = 300

In [19]:
import tensorflow as tf
tf.get_logger().setLevel('ERROR')

In [20]:
def preprocess_image(img):
    img = tf.image.decode_png(img, channels=3)
    img = tf.image.resize(img, [img_size, img_size])
    return img

In [21]:
def load_and_preprocess_image(path):
    img = tf.io.read_file(imgs_source_path + path)
    return preprocess_image(img)

In [22]:
map_dic = { class_list[i]: i for i in range(len(class_list)) }

In [23]:
X_train = train_imgs_list
X_val = test_imgs_list
Y_train = [map_dic[x.split('.')[-3]] for x in X_train]
Y_val = [map_dic[x.split('.')[-3]] for x in X_val]

In [24]:
DS_train = tf.data.Dataset.from_tensor_slices((X_train, Y_train))
DS_val = tf.data.Dataset.from_tensor_slices((X_val, Y_val))

In [25]:
def load_and_preprocess_from_path_label(path, label):
    return load_and_preprocess_image(path), tf.one_hot(label, len(class_list))

In [26]:
DS_train = DS_train.map(load_and_preprocess_from_path_label)
DS_val = DS_val.map(load_and_preprocess_from_path_label)

In [27]:
len(DS_train), len(DS_val)

(60, 15)

In [28]:
batch_size=4

In [29]:
DS_train = DS_train.shuffle(buffer_size=int(len(DS_train) * 0.8))

In [30]:
DS_batch_train = DS_train.batch(batch_size=batch_size, drop_remainder=False)
DS_batch_train = DS_batch_train.prefetch(tf.data.AUTOTUNE)
DS_batch_val = DS_val.batch(batch_size=batch_size, drop_remainder=False)

In [31]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt

In [32]:
augmentation = Sequential([
#     layers.RandomBrightness(factor=(-0.15, 0.15)),
    layers.RandomRotation(factor=(-0.15, 0.15)),
    layers.RandomTranslation(height_factor=0.1, width_factor=0.1),
    layers.RandomFlip(),
    layers.RandomContrast(factor=0.1),
], name='augmentation',)

In [33]:
n_classes = len(class_list)

In [34]:
from tensorflow.keras.applications.efficientnet import EfficientNetB3
from tensorflow.keras.applications.vgg19 import VGG19

In [45]:
def build_model(n_classes):
    model = Sequential()
    model.add(layers.Input(shape=(img_size, img_size, 3)))
    model.add(augmentation)
    
    base_model = VGG19(include_top=False, weights=None)
    base_model.trainable = True
    model.add(base_model)
    
    model.add(layers.GlobalAveragePooling2D())
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(n_classes, activation='softmax'))
    
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [46]:
model = build_model(n_classes)

In [47]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 augmentation (Sequential)   (None, 300, 300, 3)       0         
                                                                 
 vgg19 (Functional)          (None, None, None, 512)   20024384  
                                                                 
 global_average_pooling2d_1   (None, 512)              0         
 (GlobalAveragePooling2D)                                        
                                                                 
 batch_normalization_1 (Batc  (None, 512)              2048      
 hNormalization)                                                 
                                                                 
 dropout_1 (Dropout)         (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 3)                

In [48]:
epochs = 10

In [49]:
from tensorflow.keras.callbacks import ReduceLROnPlateau

In [50]:
learning_rate_reduction = ReduceLROnPlateau(
    monitor='loss',
    lr=1e-4,
    patience=2,
    verbose=1,
    factor=0.1,
    min_lr=1e-10,
)

In [51]:
callbacks = [learning_rate_reduction]

In [52]:
try:
    hist = model.fit(
        DS_batch_train, 
        epochs=epochs, 
        validation_data=DS_batch_val, 
        batch_size=batch_size, 
        shuffle=True, 
        verbose=1, 
        callbacks=callbacks
    )
except Exception as e:
    print(e)
    pass

Epoch 1/10
15/15 [==============================] - 16s 492ms/step - loss: 0.8232 - accuracy: 0.7000 - val_loss: 5.5837 - val_accuracy: 0.3333 - lr: 1.0000e-04
Epoch 2/10
15/15 [==============================] - 6s 403ms/step - loss: 0.7509 - accuracy: 0.6667 - val_loss: 9.2957 - val_accuracy: 0.3333 - lr: 1.0000e-04
Epoch 3/10
15/15 [==============================] - 6s 398ms/step - loss: 0.7966 - accuracy: 0.6333 - val_loss: 5.8625 - val_accuracy: 0.3333 - lr: 1.0000e-04
Epoch 4/10
15/15 [==============================] - ETA: 0s - loss: 0.7991 - accuracy: 0.7333
Epoch 4: ReduceLROnPlateau reducing learning rate to 9.999999747378752e-06.
15/15 [==============================] - 6s 400ms/step - loss: 0.7991 - accuracy: 0.7333 - val_loss: 4.2927 - val_accuracy: 0.3333 - lr: 1.0000e-04
Epoch 5/10
15/15 [==============================] - 6s 402ms/step - loss: 0.5173 - accuracy: 0.7667 - val_loss: 1.9444 - val_accuracy: 0.6000 - lr: 1.0000e-05
Epoch 6/10
15/15 [===========================

In [55]:
for i in range(10):
    img_test_path = random.choice(all_imgs_list)
    print(img_test_path)
    DS_test = tf.data.Dataset.from_tensor_slices([img_test_path])
    DS_test = DS_test.map(load_and_preprocess_image)
    DS_batch_test = DS_test.batch(batch_size=batch_size, drop_remainder=False)
    result = model.predict(DS_batch_test, batch_size=batch_size, max_queue_size=1, verbose=0)
    print(result)
    print(class_list[np.argmax(result[0])], '\n')

co_man_trau.18.png
[[0.0258997  0.00717248 0.96692777]]
co_man_trau 

tia_to.11.png
[[0.05137907 0.27392885 0.6746921 ]]
co_man_trau 

co_man_trau.13.png
[[8.1315773e-07 3.7801308e-06 9.9999535e-01]]
co_man_trau 

tia_to.17.png
[[0.0240629  0.00983789 0.96609926]]
co_man_trau 

co_man_trau.42.png
[[0.00920909 0.00865828 0.9821326 ]]
co_man_trau 

co_man_trau.1.png
[[0.02075404 0.00554025 0.9737057 ]]
co_man_trau 

dinh_lang.1.png
[[0.46783414 0.23227789 0.29988793]]
dinh_lang 

dinh_lang.29.png
[[0.81291    0.13255462 0.05453538]]
dinh_lang 

dinh_lang.3.png
[[0.9073656  0.04386011 0.04877424]]
dinh_lang 

dinh_lang.6.png
[[0.54529506 0.07268752 0.3820174 ]]
dinh_lang 



In [54]:
model.save('model.h5')